In [91]:
import pandas as pd
df=pd.read_excel("messages.xls", 
                  header=None, usecols=[0, 1], names=['ID', 'text'])

In [92]:
df.head()

,ID,text
0,34952194402811904,Save BBC World Service from Savage Cuts http:/...
1,34952186328784896,a lot of people always make fun about the end ...
2,34952041415581696,ReThink Group positive in outlook: Technology ...
3,34952018120409088,'Zombie' fund manager Phoenix appoints new CEO...
4,34952008683229185,Latest:: Top World Releases http://globalclass...


In [93]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [138]:
df['text_without_stopwords'] = df['text'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stop)]))
df['text_without_stopwords'] = df['text_without_stopwords'].str.replace('\d+', '')
df['text_without_stopwords']=df['text_without_stopwords'].apply(str.lower)


<ipython-input-138-9669622a4235>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text_without_stopwords'] = df['text_without_stopwords'].str.replace('\d+', '')


In [139]:
df.head()

,ID,text,text_without_stopwords,token
0,34952194402811904,Save BBC World Service from Savage Cuts http:/...,save bbc world service savage cuts http://www....,"[Save, BBC, World, Service, Savage, Cuts, http..."
1,34952186328784896,a lot of people always make fun about the end ...,lot people always make fun end world question ...,"[lot, people, always, make, fun, end, world, q..."
2,34952041415581696,ReThink Group positive in outlook: Technology ...,rethink group positive outlook: technology sta...,"[ReThink, Group, positive, outlook, Technology..."
3,34952018120409088,'Zombie' fund manager Phoenix appoints new CEO...,'zombie' fund manager phoenix appoints new ceo...,"[Zombie, fund, manager, Phoenix, appoints, new..."
4,34952008683229185,Latest:: Top World Releases http://globalclass...,latest:: top world releases http://globalclass...,"[Latest, Top, World, Releases, http, globalcla..."


In [140]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
df['token'] = df['text_without_stopwords'].apply(tokenizer.tokenize)


In [141]:
df.head()


,ID,text,text_without_stopwords,token
0,34952194402811904,Save BBC World Service from Savage Cuts http:/...,save bbc world service savage cuts http://www....,"[save, bbc, world, service, savage, cuts, http..."
1,34952186328784896,a lot of people always make fun about the end ...,lot people always make fun end world question ...,"[lot, people, always, make, fun, end, world, q..."
2,34952041415581696,ReThink Group positive in outlook: Technology ...,rethink group positive outlook: technology sta...,"[rethink, group, positive, outlook, technology..."
3,34952018120409088,'Zombie' fund manager Phoenix appoints new CEO...,'zombie' fund manager phoenix appoints new ceo...,"[zombie, fund, manager, phoenix, appoints, new..."
4,34952008683229185,Latest:: Top World Releases http://globalclass...,latest:: top world releases http://globalclass...,"[latest, top, world, releases, http, globalcla..."


In [142]:
#test to see if numbers have been removed
df['token'][26]

['bbc',
 'news',
 'charles',
 'taylor',
 'lawyer',
 'storms',
 'war',
 'crimes',
 'trial',
 'http',
 'www',
 'bbc',
 'co',
 'uk',
 'news',
 'world',
 'africa']

In [143]:
df['token'][27]

['good',
 'morning',
 'world',
 'i',
 'feel',
 'pretty',
 'happy',
 'today',
 'loving',
 'life']

In [152]:
from collections import defaultdict
def create_index (data):
        index = defaultdict(list)
        for i, tokens in enumerate(data):
            for token in tokens:
                index[token].append(i)
        return index

In [155]:
for index, row in df.iterrows():
    create_index (row['token'])

In [148]:
create_index([df['token'][26], df['token'][27]])

defaultdict(list,
            {'bbc': [0, 0],
             'news': [0, 0],
             'charles': [0],
             'taylor': [0],
             'lawyer': [0],
             'storms': [0],
             'war': [0],
             'crimes': [0],
             'trial': [0],
             'http': [0],
             'www': [0],
             'co': [0],
             'uk': [0],
             'world': [0, 1],
             'africa': [0],
             'good': [1],
             'morning': [1],
             'i': [1],
             'feel': [1],
             'pretty': [1],
             'happy': [1],
             'today': [1],
             'loving': [1],
             'life': [1]})

NameError: name 're' is not defined